In [ ]:
import os
from PIL import Image
import numpy as np
import random
import matplotlib.pyplot as plt

IMAGE_SIZE = 32

class Dataset:
    
    def __init__(self, folder):
        self.positives = self.load_folder(os.path.join(folder, "pos"))
        self.negatives = self.load_folder(os.path.join(folder, "neg"))
        
        self.last_pos = 0
        self.last_neg = 0
        
    def load_folder(self, folder):
        images = []
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            if not os.path.isfile(file_path):
                continue
                
            image = Image.open(file_path)
            image = image.convert('L').resize((IMAGE_SIZE, IMAGE_SIZE))
            images.append(np.array(image))
        return images
    
    def get_batch(self, batch_size):
        if self.last_pos + batch_size//2 > len(self.positives):
            self.last_pos = 0
            random.shuffle(self.positives)
            
        if self.last_neg + batch_size//2 > len(self.negatives):
            self.last_neg = 0
            random.shuffle(self.negatives)
        
        pos_images = self.positives[self.last_pos:(self.last_pos + batch_size//2)]
        neg_images = self.negatives[self.last_neg:(self.last_neg + batch_size//2)]
        
        pos_labels = [1] * (batch_size//2)
        neg_labels = [0] * (batch_size//2)
        
        self.last_pos = self.last_pos + batch_size//2 
        self.last_neg = self.last_neg + batch_size//2
        
        images = [item for sublist in zip(pos_images, neg_images) for item in sublist]
        data = np.dstack(images)
        data = data.transpose(2, 0, 1)
        data = data[:, np.newaxis, :, :]
        
        labels = np.array([item for sublist in zip(pos_labels, neg_labels) for item in sublist])
        labels = labels[:, np.newaxis, np.newaxis, np.newaxis]
        return data, labels
    
    def __str__(self):
        return "%d, %d" % (len(self.positives), len(self.negatives))
        
train = Dataset("/home/sebastien/Repos/ball/data/train")
val = Dataset("/home/sebastien/Repos/ball/data/val")

train.get_batch(10)

print("Train:", str(train))
print("Val:  ", str(val))

In [ ]:
import torch
from torch import nn

learning_rate = 0.00001
BATCH_SIZE = 512

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torch.nn.Sequential(
            torch.nn.Conv2d(1, 64, 5, padding=2, stride=2), # 16
            torch.nn.LeakyReLU(), # 16
            torch.nn.Conv2d(64, 128, 5, padding=2, stride=2), # 8
            torch.nn.LeakyReLU(),
            torch.nn.Conv2d(128, 1, 1, padding=1), # 16
            torch.nn.AvgPool2d(8),
            torch.nn.Sigmoid(),
        ).to(device)

def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(-0.3)

model.apply(init_weights)

loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
losses = []
for t in range(4000):
    data, labels = train.get_batch(BATCH_SIZE)
#     print(labels)
    x = torch.Tensor(data).to(device)
    y = torch.Tensor(labels).to(device)
    
#     print(x.shape, y.shape)
    
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)
#     print(y_pred)
#     print(y_pred[0, 0, 0, 0].item())

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
#     print(loss.item())
    losses.append(loss.item())
#     print(y_pred.shape)

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the Tensors it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its parameters
    optimizer.step()
    
fig, plot = plt.subplots(1, 1, figsize=(15, 15))
plot.plot(range(len(losses)), losses, color="blue")

In [ ]:
tp = 0
fp = 0 
fn = 0
tn = 0

threshold = 0.7

for pos in val.positives:
    score = model(torch.Tensor(np.array(pos)[np.newaxis, np.newaxis, :, :]).to(device)).item()
    
    if score > threshold:
        tp += 1
    else:
        fn += 1

for neg in val.negatives:
    score = model(torch.Tensor(np.array(neg)[np.newaxis, np.newaxis, :, :]).to(device)).item()
    
    if score > threshold:
        fp += 1
    else:
        tn += 1
        
print("Accuracy:  %.2f" % ((tp + tn) / (tp + tn + fn + fp)))
print("Recall:    %.2f" % (tp / (tp + fn)))
print("Precision: %.2f" % (tp / (tp + fp)))

In [ ]:

predictions = []
for data in val.positives:
    score = model(torch.Tensor(np.array(data)[np.newaxis, np.newaxis, :, :]).to(device)).item()
    predictions.append([score, data])
    
predictions = sorted(predictions, key=lambda x: x[0])

for prediction in predictions[:20]:
    score, data = prediction
    fig, plot = plt.subplots(1, 1)
    plot.imshow(data, cmap="gray")
    plot.set_title("%.2f" % score)

In [ ]:
predictions = []
for data in val.negatives:
    score = model(torch.Tensor(np.array(data)[np.newaxis, np.newaxis, :, :]).to(device)).item()
    predictions.append([score, data])
    
predictions = sorted(predictions, key=lambda x: x[0], reverse=True)

for prediction in predictions[:100]:
    score, data = prediction
    fig, plot = plt.subplots(1, 1)
    plot.imshow(data, cmap="gray")
    plot.set_title("%.2f" % score)